In [2]:
import numpy as np

from nnunetv2.evaluation.evaluate_predictions import region_or_label_to_mask
from acvl_utils.morphology.morphology_helper import remove_all_but_largest_component

from typing import Union, Tuple, List

In [4]:
def remove_all_but_largest_component_from_segmentation(
    segmentation: np.ndarray,
    labels_or_regions: Union[int, Tuple[int, ...], List[Union[int, Tuple[int, ...]]]],
    background_label: int = 0,
) -> np.ndarray:
    mask = np.zeros_like(segmentation, dtype=bool)
    if not isinstance(labels_or_regions, list):
        labels_or_regions = [labels_or_regions]
    for l_or_r in labels_or_regions:
        mask |= region_or_label_to_mask(segmentation, l_or_r)
    mask_keep = remove_all_but_largest_component(mask)
    ret = np.copy(segmentation)  # do not modify the input!
    ret[mask & ~mask_keep] = background_label
    return ret

### 2. Segmentation Refinement Function

This function combines Otsu's thresholding with the largest connected component analysis to refine a coarse initial mask (like a bounding box) into a precise segmentation.

In [5]:
import SimpleITK as sitk
from skimage.filters import threshold_otsu

def refine_segmentation_with_otsu(
    image_sitk: sitk.Image, 
    initial_mask_sitk: sitk.Image
) -> sitk.Image:
    """
    Refines a segmentation mask using Otsu's thresholding and largest connected component analysis.
    This version accepts and returns SimpleITK images to preserve metadata.

    Args:
        image_sitk (sitk.Image): The grayscale SimpleITK image (e.g., CT or PET).
        initial_mask_sitk (sitk.Image): A SimpleITK binary mask indicating the region of interest (e.g., a bounding box).

    Returns:
        sitk.Image: The refined binary segmentation mask as a SimpleITK image.
    """
    # Convert SimpleITK images to NumPy arrays for processing
    # Note: sitk.GetArrayFromImage swaps the axis order (Z, Y, X)
    image_np = sitk.GetArrayFromImage(image_sitk)
    initial_mask_np = sitk.GetArrayFromImage(initial_mask_sitk)

    # Ensure the initial mask is boolean
    initial_mask_np = initial_mask_np.astype(bool)
    
    # Step 1: Apply Otsu's thresholding ONLY to the region of interest
    pixels_in_roi = image_np[initial_mask_np]
    
    if pixels_in_roi.size == 0:
        # Return an empty sitk image with correct metadata if the mask is empty
        refined_mask_np = np.zeros_like(image_np, dtype=np.uint8)
    else:
        otsu_thresh = threshold_otsu(pixels_in_roi)
        
        # Step 2: Create a binary mask and limit it to the initial ROI
        binary_mask = image_np > otsu_thresh
        binary_mask[~initial_mask_np] = 0
        
        # Step 3: Keep only the largest connected component
        refined_mask_np = remove_all_but_largest_component_from_segmentation(binary_mask, labels_or_regions=1)
        refined_mask_np = refined_mask_np.astype(np.uint8)

    # Convert the resulting NumPy array back to a SimpleITK image
    refined_mask_sitk = sitk.GetImageFromArray(refined_mask_np)
    
    # Copy metadata (origin, spacing, direction) from the original image
    refined_mask_sitk.CopyInformation(image_sitk)
    
    return refined_mask_sitk


In [ ]:
import os 

root_dir = '../../data/0_nifti'
subject_id = 'sub-AKHABDALLAADELAHMED20091023'
out_dir = root_dir

pet_path = os.path.join(root_dir, subject_id, f'{subject_id}_pet.nii.gz')
lesion_mask_path = os.path.join(root_dir, subject_id, f'{subject_id}_seg-lesion.nii.gz')

if os.path.exists(pet_path) and os.path.exists(lesion_mask_path):
    print(f'Processing {subject_id}...')
    pet_sitk = sitk.ReadImage(pet_path)
    lesion_mask_sitk = sitk.ReadImage(lesion_mask_path)

    refined_mask_sitk = refine_segmentation_with_otsu(pet_sitk, lesion_mask_sitk)

    os.makedirs(os.path.join(out_dir, subject_id), exist_ok=True)
    refined_mask_path = os.path.join(out_dir, subject_id, f'{subject_id}_seg-lesion-refined.nii.gz')
    sitk.WriteImage(refined_mask_sitk, refined_mask_path)

else:
    print(f'Files for {subject_id} not found. Skipping...')

Processing sub-AKHABDALLAADELAHMED20091023...


In [ ]:
import os 
from tqdm import tqdm

root_dir = '../../data/0_nifti'
subject_ids = os.listdir(root_dir)

pbar = tqdm(subject_ids, desc='Refining segmentations')

for subject_id in pbar:
    pbar.set_description(f'Processing {subject_id}')

    pet_path = os.path.join(root_dir, subject_id, f'{subject_id}_pet.nii.gz')
    lesion_mask_path = os.path.join(root_dir, subject_id, f'{subject_id}_seg-lesion.nii.gz')

    if os.path.exists(pet_path) and os.path.exists(lesion_mask_path):
        print(f'Processing {subject_id}...')
        pet_sitk = sitk.ReadImage(pet_path)
        lesion_mask_sitk = sitk.ReadImage(lesion_mask_path)

        refined_mask_sitk = refine_segmentation_with_otsu(pet_sitk, lesion_mask_sitk)

        os.makedirs(os.path.join(root_dir, subject_id), exist_ok=True)
        refined_mask_path = os.path.join(root_dir, subject_id, f'{subject_id}_seg-lesion-refined.nii.gz')
        sitk.WriteImage(refined_mask_sitk, refined_mask_path)

    else:
        tqdm.write(f'Files for {subject_id} not found. Skipping...')

pbar.set_description('Processing complete.')
pbar.close()

  0%|          | 0/1030 [00:00<?, ?it/s]

Processing sub-NM136...


  0%|          | 1/1030 [00:00<08:09,  2.10it/s]

Processing sub-AKHKUNTNERVERA20230123...


  0%|          | 2/1030 [00:08<1:21:17,  4.74s/it]

...

Processing sub-NM124...


 94%|█████████▍| 970/1030 [10:14<00:18,  3.23it/s]

Processing sub-AKHPECKAREINHARDING20171116...


 94%|█████████▍| 971/1030 [10:14<00:18,  3.26it/s]

Processing sub-AKHPAULKURT20240821...
